# Model 2: High resolution prediction of flood maps

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import shutil

Loading of the previously computed predictions

In [ ]:
predictions = xr.open_dataset("localdata/Model2_Score_Full_Rez_inf.nc")

In [ ]:
relevent_data_tag_binary_mask = xr.open_dataset("localdata/raw/relevent_data_tag_binary_mask.nc")

nan_mask = relevent_data_tag_binary_mask["__xarray_dataarray_variable__"] == 1
data_array = predictions["M2_score"]

for time in predictions.time.values:
    time_slice = data_array.sel(time=time)
    time_slice = time_slice.where(~nan_mask, -1)
    data_array.loc[dict(time=time)] = time_slice

predictions["M2_score"] = data_array
predictions.to_netcdf(f'localdata/predictions_tmp.nc', engine='h5netcdf')


In [ ]:
np.unique(predictions["M2_score"].values)

In [ ]:
def from_xarray_to_vector(data: xr.DataArray):
    data = data.sortby("time").sortby("x").sortby("y")

    xry = data.M2_score.values
    vectors = xry.reshape(xry.shape[0], xry.shape[1]*xry.shape[2])
    vector = vectors.flatten()
    mask = vector == -1
    return vector[~mask]

**The submission file should be a flatten vector**

Conversion to vector

In [ ]:
out = from_xarray_to_vector(predictions)

Sanity check

In [ ]:
len(out) # it should be 112923207

In [ ]:
pd.DataFrame(out).to_csv("localdata/pred.csv")
shutil.make_archive("localdata/pred.csv", 'zip', "localdata","pred.csv")